In [62]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Does nn.Conv2d init work well?

[Jump_to lesson 9 video](https://course.fast.ai/videos/?lesson=9&t=21)

In [63]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): 
    return (x-m)/s

In [64]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/modules/conv.py
Type:      function


In [65]:
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [66]:
x_train = x_train.view(-1,1,28,28)
x_valid = x_valid.view(-1,1,28,28)
x_train.shape,x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [67]:
n,*_ = x_train.shape
c = y_train.max()+1
nh = 32
n,c

(50000, tensor(10))

In [68]:
l1 = nn.Conv2d(1, nh, 5)

In [69]:
x = x_valid[:100]

In [70]:
x.shape

torch.Size([100, 1, 28, 28])

In [71]:
def stats(x): 
    return x.mean(),x.std()

In [72]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [73]:
stats(l1.weight),stats(l1.bias)

((tensor(0.0009, grad_fn=<MeanBackward0>),
  tensor(0.1157, grad_fn=<StdBackward0>)),
 (tensor(-0.0040, grad_fn=<MeanBackward0>),
  tensor(0.1098, grad_fn=<StdBackward0>)))

In [74]:
t = l1(x)

In [75]:
stats(t)

(tensor(-0.0023, grad_fn=<MeanBackward0>),
 tensor(0.6931, grad_fn=<StdBackward0>))

In [76]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1(x))

(tensor(-0.0316, grad_fn=<MeanBackward0>),
 tensor(1.1046, grad_fn=<StdBackward0>))

In [77]:
import torch.nn.functional as F

In [78]:
def f1(x,a=0): 
    return F.leaky_relu(l1(x),a)

In [79]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5365, grad_fn=<MeanBackward0>),
 tensor(0.9987, grad_fn=<StdBackward0>))

In [80]:
l1 = nn.Conv2d(1, nh, 5)
stats(f1(x))

(tensor(0.2247, grad_fn=<MeanBackward0>),
 tensor(0.4012, grad_fn=<StdBackward0>))

In [81]:
l1.weight.shape

torch.Size([32, 1, 5, 5])

In [82]:
# receptive field size
rec_fs = l1.weight[0,0].numel()
rec_fs

25

In [95]:
?l1

Signature:      l1(*input, **kwargs)
Type:           Conv2d
String form:    Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
File:           ~/anaconda3/envs/py38/lib/python3.8/site-packages/torch/nn/modules/conv.py
Docstring:     
Applies a 2D convolution over an input signal composed of several input
planes.

In the simplest case, the output value of the layer with input size
:math:`(N, C_{\text{in}}, H, W)` and output :math:`(N, C_{\text{out}}, H_{\text{out}}, W_{\text{out}})`
can be precisely described as:

.. math::
    \text{out}(N_i, C_{\text{out}_j}) = \text{bias}(C_{\text{out}_j}) +
    \sum_{k = 0}^{C_{\text{in}} - 1} \text{weight}(C_{\text{out}_j}, k) \star \text{input}(N_i, k)


where :math:`\star` is the valid 2D `cross-correlation`_ operator,
:math:`N` is a batch size, :math:`C` denotes a number of channels,
:math:`H` is a height of input planes in pixels, and :math:`W` is
width in pixels.

* :attr:`stride` controls the stride for the cross-correlation, a single
  number 

In [21]:
nf,ni,*_ = l1.weight.shape
nf,ni

(32, 1)

In [22]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [23]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [24]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [25]:
torch.zeros(10000).uniform_(-1,1).std()

tensor(0.5803)

In [26]:
1/math.sqrt(3.)

0.5773502691896258

In [27]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [28]:
kaiming2(l1.weight, a=0);
stats(f1(x))

(tensor(0.5608, grad_fn=<MeanBackward0>),
 tensor(1.1550, grad_fn=<StdBackward0>))

In [29]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))

(tensor(0.2120, grad_fn=<MeanBackward0>),
 tensor(0.3805, grad_fn=<StdBackward0>))

In [30]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

In [31]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [32]:
y = y_valid[:100].float()

In [33]:
t = m(x)
stats(t)

(tensor(-0.0420, grad_fn=<MeanBackward0>),
 tensor(0.0117, grad_fn=<StdBackward0>))

In [34]:
l = mse(t,y)
l.backward()

In [35]:
stats(m[0].weight.grad)

(tensor(0.0203), tensor(0.0501))

In [36]:
init.kaiming_uniform_??

Signature:
init.kaiming_uniform_(
    tensor,
    a=0,
    mode='fan_in',
    nonlinearity='leaky_relu',
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in `Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification` - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \text{gain} \times \sqrt{\frac{3}{\text{fan\_mode}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (only 
        used with ``'leaky_relu'``)
        mode: either ``'fan_in'`` (default) or ``'fan_out'``. Choosing ``'fan_in'``
            preserves the magnitude of the variance of the weights in

In [37]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [38]:
t = m(x)
stats(t)

(tensor(1.2138, grad_fn=<MeanBackward0>),
 tensor(0.3149, grad_fn=<StdBackward0>))

In [39]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)

(tensor(-0.0614), tensor(0.6389))

## Export

In [40]:
!./notebook2script.py 02a_why_sqrt5.ipynb

Converted 02a_why_sqrt5.ipynb to exp/nb_02a.py
